In [1]:
import numpy as np
import pandas as pd
from gurobipy import *

In [2]:
df = pd.read_csv('Data/Data_DJIA.csv', index_col = 1)
df['ADJ_PRC'] =  df['PRC']/df['CFACPR']
price = df.pivot(columns = 'PERMNO', values = 'ADJ_PRC')

In [3]:
code_dict = {24643: 'Alcoa',
             59176: 'Amex',
             19561: 'Boeing',
             14541: 'Chev',
             11308: 'Coke',
             11703: 'Du Pont',
             22592: 'MMM',
             18163: 'P&G',
             14322: 'Sears',
             17830: 'U Tech'}

In [4]:
price.rename(columns = code_dict, inplace = True)
price = price[code_dict.values()]

In [5]:
rtn = price.pct_change()[2:]

In [6]:
mean = rtn.mean().values
std = rtn.std().values
corr = rtn.corr().values
cov = rtn.cov().values

In [7]:
rtn.mean()

PERMNO
Alcoa      0.010971
Amex       0.016781
Boeing     0.013849
Chev       0.010020
Coke       0.017898
Du Pont    0.011673
MMM        0.011580
P&G        0.013074
Sears      0.009936
U Tech     0.010003
dtype: float64

In [8]:
rtn.std()

PERMNO
Alcoa      0.086669
Amex       0.084585
Boeing     0.097071
Chev       0.084926
Coke       0.060055
Du Pont    0.068438
MMM        0.057920
P&G        0.056477
Sears      0.080176
U Tech     0.081194
dtype: float64

In [17]:
#Build basic portfolio
t = 50
a = 2/t
N = len(mean)
stocks = list(code_dict.values())

m = Model("Basic Portfolio")
w = pd.Series(m.addVars(stocks, lb = 0, ub = GRB.INFINITY), index=stocks)
m.update()

#Set Objective Function
obj = mean.dot(w) - 0.5*a*cov.dot(w).dot(w)
m.setObjective(obj,GRB.MAXIMIZE)
m.addConstr(w.sum() == 1, 'budget')
m.update()

m.setParam('OutputFlag', 0)
m.optimize()

CE_0 = m.ObjVal

In [18]:
[w[i].x for i in range(len(w))]

[2.9971220721860034e-09,
 3.6158324464088953e-07,
 8.661114276293456e-09,
 5.557975448248265e-09,
 0.9999995866182253,
 7.40093388989199e-09,
 7.253477221330161e-09,
 8.49802493416938e-09,
 5.6529362008952085e-09,
 5.776946270697359e-09]

In [11]:
#Parameter Setting
k_list = np.arange(0.05, 0.20, 0.05)
iter_num = 100

In [19]:
#Examine the effect of errors in means
for k in k_list:
    CEL = 0
    for n in range(iter_num):
        z = np.random.normal(size = N)*k + 1
        mean_new = np.multiply(mean, z)
    
        m = Model("Mean Sensitivity")
        w = pd.Series(m.addVars(stocks, lb = 0, ub = GRB.INFINITY), index=stocks)
        m.update()

        #Set Objective Function
        obj = mean_new.dot(w) - 0.5*a*cov.dot(w).dot(w)
        m.setObjective(obj,GRB.MAXIMIZE)
        m.addConstr(w.sum() == 1, 'budget')
        m.update()
        
        m.setParam('OutputFlag', 0)
        m.optimize()
        
        w = np.array([w[i].x for i in range(len(w))])
        CE_x = np.dot(mean,w)- 0.5*a*np.dot(np.dot(w.T, cov), w)
        CEL += (1 - CE_x/CE_0)
    
    CEL /= iter_num
    print(CEL)

0.012197833536142259
0.02361336832000739
0.03599023388973877
0.06384199706430724


In [20]:
#Examine the effect of errors in variances
for k in k_list:
    CEL = 0
    n = 0
    while n < iter_num:
        try:
            z = np.random.normal(size = N)*k + 1
            var = np.diag(cov)
            var_new = np.multiply(var, z)
            cov_new = cov + np.diag(var_new - var)
    
            m = Model("Variance Sensitivity")
            w = pd.Series(m.addVars(stocks, lb = 0, ub = GRB.INFINITY), index=stocks)
            m.update()

            #Set Objective Function
            obj = mean.dot(w) - 0.5*a*cov_new.dot(w).dot(w)
            m.setObjective(obj,GRB.MAXIMIZE)
            m.addConstr(w.sum() == 1, 'budget')
            m.update()
        
            m.setParam('OutputFlag', 0)
            m.optimize()
        
            w = np.array([w[i].x for i in range(len(w))])
            CE_x = np.dot(mean,w)- 0.5*a*np.dot(np.dot(w.T, cov), w)
            CEL += (1 - CE_x/CE_0)
            n += 1
        except GurobiError:
            continue
    
    CEL /= iter_num
    print(CEL)

3.02668885288071e-08
6.538079083906467e-08
1.2466311328451773e-07
1.1876801311649921e-07


In [21]:
#Examine the effect of errors in variances
for k in k_list:
    CEL = 0
    n = 0
    while n < iter_num:
        try:
            z = np.random.normal(size = N**2)*k + 1
            var = np.diag(cov)
            upper = np.triu(cov)
            cov_new = np.multiply(upper.reshape(-1), z).reshape((N,N))
            cov_new += cov_new.T
            cov_new = cov_new - np.diag(np.diag(cov_new)) + np.diag(var)
    
            m = Model("Variance Sensitivity")
            w = pd.Series(m.addVars(stocks, lb = 0, ub = GRB.INFINITY), index=stocks)
            m.update()

            #Set Objective Function
            obj = mean.dot(w) - 0.5*a*cov_new.dot(w).dot(w)
            m.setObjective(obj,GRB.MAXIMIZE)
            m.addConstr(w.sum() == 1, 'budget')
            m.update()
        
            m.setParam('OutputFlag', 0)
            m.optimize()
        
            w = np.array([w[i].x for i in range(len(w))])
            CE_x = np.dot(mean,w)- 0.5*a*np.dot(np.dot(w.T, cov), w)
            CEL += (1 - CE_x/CE_0)
            n += 1
        except GurobiError:
            continue
            
    CEL /= iter_num
    print(CEL)

1.1286981445035593e-08
3.591907138722483e-08
6.941143194660882e-08
5.912635483107387e-08
